In [ ]:
# ---------- INSTALLATION AND IMPORTATION ----------

!pip install googleapis-common-protos protobuf grpcio pandas systemathics.apis
!pip install sklearn
!pip install kneed
!pip install statsmodels

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import AffinityPropagation
import scipy.cluster.hierarchy as shc
from itertools import cycle
from sklearn import metrics
from kneed import KneeLocator
import numpy as np
import pandas as pd
import pair_selection
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
s = pair_selection.Selection(start_date="2015-01-01") # adjustment=False

In [ ]:
s.df_all_prices

In [ ]:
data = s.df_all_prices
data1 = data
data.set_index('Dates', inplace=True)
data

In [ ]:
# pd.set_option('precision', 3)
data.describe().T.head(10)

In [ ]:
data.isnull().values.any()

In [ ]:
#Calculate returns and create a data frame
returns = data.pct_change().mean()*266
returns = pd.DataFrame(returns)
returns.columns = ['returns']

#Calculate the volatility
returns['volatility'] = data.pct_change().std()*np.sqrt(266)

data = returns
data.head()

In [ ]:
#Prepare the scaler
scale = StandardScaler().fit(data)

#Fit the scaler
scaled_data = pd.DataFrame(scale.fit_transform(data), columns=data.columns, index=data.index)
X = scaled_data
X.head()

# KMeans Clustering

In [ ]:
K = range(1,15)
distortions = []

#Fit the method
for k in K:
    kmeans = KMeans(n_clusters = k)
    kmeans.fit(X)
    distortions.append(kmeans.inertia_)

#Plot the results
fig = plt.figure(figsize= (15,5))
plt.plot(K, distortions, 'bx-')
plt.xlabel('Values of K')
plt.ylabel('Distortion')
plt.title('Elbow Method')
plt.grid(True)
plt.show()

In [ ]:
kl = KneeLocator(K, distortions, curve="convex", direction="decreasing")
kl.elbow

In [ ]:
#For the silhouette method k needs to start from 2
K = range(2,15)
silhouettes = []

#Fit the method
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10, init='random')
    kmeans.fit(X)
    silhouettes.append(silhouette_score(X, kmeans.labels_))

#Plot the results
fig = plt.figure(figsize=(15,5))
plt.plot(K, silhouettes, 'bx-')
plt.xlabel('Values of K')
plt.ylabel('Silhouette score')
plt.title('Silhouette Method')
plt.grid(True)
plt.show()

kl = KneeLocator(K, silhouettes, curve="convex", direction="decreasing")
print('Suggested number of clusters: ', kl.elbow)

In [ ]:
c = 4
#Fit the model
k_means = KMeans(n_clusters=c)
k_means.fit(X)
prediction = k_means.predict(X)

#Plot the results
centroids = k_means.cluster_centers_
fig = plt.figure(figsize = (18,10))
ax = fig.add_subplot(111)
scatter = ax.scatter(X.iloc[:,0],X.iloc[:,1], c=k_means.labels_, cmap="rainbow", label = X.index)
ax.set_title('k-Means Cluster Analysis Results')
ax.set_xlabel('Mean Return')
ax.set_ylabel('Volatility')
plt.colorbar(scatter)
plt.plot(centroids[:,0],centroids[:,1],'sg',markersize=10)
plt.show()

In [ ]:
clustered_series = pd.Series(index=X.index, data=k_means.labels_.flatten())
clustered_series_all = pd.Series(index=X.index, data=k_means.labels_.flatten())
clustered_series = clustered_series[clustered_series != -1]
plt.figure(figsize=(12,8))
plt.barh(range(len(clustered_series.value_counts())),clustered_series.value_counts())
plt.title('Clusters')
plt.xlabel('Stocks per Cluster')
plt.ylabel('Cluster Number')
plt.show()

# Hierarchical Clustering

In [ ]:
plt.figure(figsize=(15, 10))  
plt.title("Dendrograms")  
dend = shc.dendrogram(shc.linkage(X, method='ward'))


In [ ]:
plt.figure(figsize=(15, 10))  
plt.title("Dendrogram")  
dend = shc.dendrogram(shc.linkage(X, method='ward'))
plt.axhline(y=13.5, color='purple', linestyle='--')

In [ ]:
#Fit the model
clusters = 4
hc = AgglomerativeClustering(n_clusters= clusters, affinity='euclidean', linkage='ward')
labels = hc.fit_predict(X)

#Plot the results
fig = plt.figure(figsize=(15,10))
ax = fig.add_subplot(111)
scatter = ax.scatter(X.iloc[:,0], X.iloc[:,1], c=labels, cmap='rainbow')
ax.set_title('Hierarchical Clustering Results')
ax.set_xlabel('Mean Return')
ax.set_ylabel('Volatility')
plt.colorbar(scatter)
plt.show()

# Affinity Propagation Clustering

In [ ]:
ap = AffinityPropagation()
ap.fit(X)
labels1 = ap.predict(X)

#Plot the results
fig = plt.figure(figsize=(15,10))
ax = fig.add_subplot(111)
scatter = ax.scatter(X.iloc[:,0], X.iloc[:,1], c=labels1, cmap='rainbow')
ax.set_title('Affinity Propagation Clustering Results')
ax.set_xlabel('Mean Return')
ax.set_ylabel('Volatility')
plt.colorbar(scatter)
plt.show()

In [ ]:
#Extract the cluster centers and labels
cci = ap.cluster_centers_indices_
labels2 = ap.labels_

#Print their number
clusters = len(cci)
print('The number of clusters is:',clusters)

#Plot the results
X_ap = np.asarray(X)
plt.close('all')
plt.figure(1)
plt.clf
fig=plt.figure(figsize=(15,10))
colors = cycle('cmykrgbcmykrgbcmykrgbcmykrgb')
for k, col in zip(range(clusters),colors):
    cluster_members = labels2 == k
    cluster_center = X_ap[cci[k]]
    plt.plot(X_ap[cluster_members, 0], X_ap[cluster_members, 1], col + '.')
    plt.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col, markeredgecolor='k', markersize=12)
    for x in X_ap[cluster_members]:
        plt.plot([cluster_center[0], x[0]], [cluster_center[1], x[1]], col)

plt.show()

# Compare clustering models

In [ ]:
print("k-Means Clustering", metrics.silhouette_score(X, k_means.labels_, metric='euclidean'))
print("Hierarchical Clustering", metrics.silhouette_score(X, hc.fit_predict(X), metric='euclidean'))
print("Affinity Propagation Clustering", metrics.silhouette_score(X, ap.labels_, metric='euclidean'))

# Extract the trading pairs

In [ ]:
cluster_size_limit = 1000
counts = clustered_series.value_counts()
ticker_count = counts[(counts>1) & (counts<=cluster_size_limit)]
print ("Number of clusters: %d" % len(ticker_count))
print ("Number of Pairs: %d" % (ticker_count*(ticker_count-1)).sum())

In [ ]:
def find_cointegrated_pairs(data, significance=0.05):
    n = data.shape[1]    
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = data.keys()
    pairs = []
    for i in range(1):
        for j in range(i+1, n):
            S1 = data[keys[i]]            
            S2 = data[keys[j]]
            result = coint(S1, S2)
            score = result[0]
            pvalue = result[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < significance:
                pairs.append((keys[i], keys[j]))
    return score_matrix, pvalue_matrix, pairs

In [ ]:
from statsmodels.tsa.stattools import coint

cluster_dict = {}

for i, clust in enumerate(ticker_count.index):
    tickers = clustered_series[clustered_series == clust].index
    score_matrix, pvalue_matrix, pairs = find_cointegrated_pairs(data1[tickers])
    cluster_dict[clust] = {}
    cluster_dict[clust]['score_matrix'] = score_matrix
    cluster_dict[clust]['pvalue_matrix'] = pvalue_matrix
    cluster_dict[clust]['pairs'] = pairs
    
pairs = []   
for cluster in cluster_dict.keys():
    pairs.extend(cluster_dict[cluster]['pairs'])
    
print ("Number of pairs:", len(pairs))
print ("In those pairs, we found %d unique tickers." % len(np.unique(pairs)))
print(pairs)

___

# Get best pairs function

In [ ]:
from statsmodels.tsa.stattools import coint

In [ ]:
s = pair_selection.Selection(start_date="2015-01-01") # adjustment=False

In [ ]:
def get_best_pairs(df_prices, start_date='', end_date=''):
    # Retrieve only the prices between the two specified dates
    mask = (df_prices['Dates'] >= start_date) & (df_prices['Dates'] <= end_date)
    data = df_prices.loc[mask]
    
    data.set_index('Dates', inplace=True)
    data1 = data.copy(deep=True)
    
    #Calculate returns and create a data frame
    returns = data.pct_change().mean()*266
    returns = pd.DataFrame(returns)
    returns.columns = ['returns']

    #Calculate the volatility
    returns['volatility'] = data.pct_change().std()*np.sqrt(266)

    data = returns
    
    #Prepare the scaler
    scale = StandardScaler().fit(data)

    #Fit the scaler
    scaled_data = pd.DataFrame(scale.fit_transform(data), columns=data.columns, index=data.index)
    X = scaled_data
    
    K = range(1,15)
    distortions = []

    #Fit the method
    for k in K:
        kmeans = KMeans(n_clusters = k)
        kmeans.fit(X)
        distortions.append(kmeans.inertia_)
    
    kl = KneeLocator(K, distortions, curve="convex", direction="decreasing")
    c = kl.elbow

    #Fit the model
    k_means = KMeans(n_clusters=c)
    k_means.fit(X)
    prediction = k_means.predict(X)
    
    clustered_series = pd.Series(index=X.index, data=k_means.labels_.flatten())
    clustered_series_all = pd.Series(index=X.index, data=k_means.labels_.flatten())
    clustered_series = clustered_series[clustered_series != -1]
    
    cluster_size_limit = 1000
    counts = clustered_series.value_counts()
    ticker_count = counts[(counts>1) & (counts<=cluster_size_limit)]
    
    cluster_dict = {}

    for i, clust in enumerate(ticker_count.index):
        tickers = clustered_series[clustered_series == clust].index
        score_matrix, pvalue_matrix, pairs = find_cointegrated_pairs(data1[tickers])
        cluster_dict[clust] = {}
        cluster_dict[clust]['score_matrix'] = score_matrix
        cluster_dict[clust]['pvalue_matrix'] = pvalue_matrix
        cluster_dict[clust]['pairs'] = pairs

    pairs = []   
    for cluster in cluster_dict.keys():
        pairs.extend(cluster_dict[cluster]['pairs'])

    # print ("Number of pairs:", len(pairs))
    # print ("In those pairs, we found %d unique tickers." % len(np.unique(pairs)))
    # print(pairs)
    return pairs

In [ ]:
prices = s.df_all_prices
top = get_best_pairs(prices, "2015-01-01", "2022-01-23")

In [ ]:
top

In [ ]:
l = [[top[i][0], top[i][1]] for i in range(len(top))]
l

# Get all time best pairs

In [ ]:
from statsmodels.tsa.stattools import coint
from dateutil.relativedelta import relativedelta
from datetime import datetime, date
import json

In [ ]:
start_date="2015-01-01"
interval=6
repetition=1
filename="ml_best_pairs"

In [ ]:
s = pair_selection.Selection(start_date=start_date) 

In [ ]:
def add_months(start_date, interval):
    """From a starting date in string format 'YYYY-MM-DD', return the same format date after an interval of X month(s) later."""
    date_format = '%Y-%m-%d'
    dtObj = datetime.strptime(start_date, date_format)
    # Add months to a given datetime object
    future_date = dtObj + relativedelta(months=interval)
    # Convert datetime object to string in required format
    future_date_str = future_date.strftime(date_format)
    return future_date_str

In [ ]:
end_date = add_months(start_date, interval)
prices = s.df_all_prices

# To know when to stop the loop: the month and year of the last saved price
last_date = prices.iloc[-1]['Dates'] 
last_year = last_date.strftime('%Y')
last_month = last_date.strftime('%m')

best_pairs_dict = {}
while True:
    # Get the best pairs of a specific period
    top = get_best_pairs(prices, start_date, end_date)
    best_pairs_dict[end_date] = [[top[i][0], top[i][1]] for i in range(len(top))]
    # increment start and end for the get_best_pairs() computation
    start_date = add_months(start_date, repetition)
    end_date = add_months(end_date, repetition)
    # check if we've reached the month and year of the last saved price
    splt = end_date.split('-')    
    if splt[0] == last_year and splt[1] == last_month:
        break

# Save the results in a json for backtesting purpose
with open(filename + ".json", "w") as f:
    json.dump(best_pairs_dict, f)